In [4]:
!pip install langchain langchain-community faiss-cpu sentence-transformers openai pypdf

In [6]:
from langchain_community.document_loaders import PyPDFLoader

# Step 2: Load your document (replace with your file name)
loader = PyPDFLoader("ai-report.pdf")
documents = loader.load()

print(f"Total pages loaded: {len(documents)}")
print(documents[0].page_content[:500])  # preview first 500 characters

Total pages loaded: 71
1 
 
 
Artificial Intelligence 
and the Future of 
Teaching and Learning 
Insights and Recommendations 
May 2023


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Step 3a: Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(docs)}")

# Step 3b: Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 3c: Store in FAISS vector database
vectorstore = FAISS.from_documents(docs, embeddings)

print("✅ Documents embedded and stored in FAISS database!")

Total chunks created: 283


/tmp/ipython-input-3808130348.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Documents embedded and stored in FAISS database!


In [8]:
# === Option B: Use a small local HuggingFace model ===
# Install transformers if you haven't: !pip install transformers accelerate
# This uses a small sequence-to-sequence model (fast but limited).

from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA

model_name = "google/flan-t5-small"  # small & fast (good for testing)

# load HF pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
text_gen = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# wrap as langchain LLM
hf_llm = HuggingFacePipeline(pipeline=text_gen)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
qa_chain_hf = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff", retriever=retriever)

# Interactive loop
print("Ask questions (type 'exit' to stop):")
while True:
    q = input("\nQuestion: ")
    if q.strip().lower() in ["exit", "quit"]:
        break
    resp = qa_chain_hf.run(q)
    print("\nAnswer:\n", resp)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-367732751.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline=text_gen)


Ask questions (type 'exit' to stop):

Question: what is applications of ai?


/tmp/ipython-input-367732751.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = qa_chain_hf.run(q)
Token indices sequence length is longer than the specified maximum sequence length for this model (731 > 512). Running this sequence through the model will result in indexing errors



Answer:
 3).

Question: what is the role of ml?

Answer:
 a whole curricular approach might include roles for teachers that emphasize mathematical practices like argumentation and modeling

Question: exit


In [9]:
# Step 5: Ask a test question from your document
query = "What is the role of Machine Learning?"
response = qa_chain_hf.run(query)
print("Q:", query)
print("A:", response)

Q: What is the role of Machine Learning?
A: Automatically assess scientific models


In [10]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Step 6: Add citations & page numbers

# Custom prompt
prompt_template = """
You are an AI assistant answering questions from a document.
Use the retrieved context below to answer the question clearly and concisely.
If you don't know the answer, say "I don't know".

Context:
{context}

Question:
{question}

Answer (with page reference if available):
"""

QA_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Build QA chain with sources
qa_chain_with_sources = RetrievalQA.from_chain_type(
    llm=hf_llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": QA_PROMPT},
    return_source_documents=True
)

# Example question (you can change this)
query = "What are the applications of Artificial Intelligence?"
result = qa_chain_with_sources(query)

print("Q:", query)
print("\nAnswer:", result['result'])
print("\nSources:")
for doc in result['source_documents']:
    print(f"- Page {doc.metadata.get('page_number', 'N/A')}: {doc.page_content[:200]}...")

/tmp/ipython-input-1467875688.py:37: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain_with_sources(query)


Q: What are the applications of Artificial Intelligence?

Answer: a human-centered partnership model of people and artificial intelligence (AI) working together to enhance cognitive performance, including learning, decision making, and new experiences

Sources:
- Page N/A: 3 
As seen in voice assistants, mapping tools, shopping recommendations, essay-writing capabilities, 
and other familiar applications, AI may enhance educational services.  
Second, urgency and import...
- Page N/A: 4 White House Office of Science and Technology Policy (October 2022), Blueprint for an AI bill of rights: Making automated 
systems work for the American people. The White House Office of Science and ...
- Page N/A: important now. Today, for example, if an ITS specializes in feedback as a student practices, a 
human teacher could still be responsible for motivating student engagement and self-regulation 
along wi...
- Page N/A: 14 
Perspective: Intelligence Augmentation 
“Augmented intelligence is a desig

In [11]:
import gradio as gr

def chatbot(query):
    result = qa_chain_with_sources(query)
    answer = result['result']
    sources = "\n".join(
        [f"- Page {doc.metadata.get('page_number', 'N/A')}" for doc in result['source_documents']]
    )
    return f"{answer}\n\nSources:\n{sources}"

# Create Gradio interface
demo = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Ask a question about your PDF"),
    outputs=gr.Textbox(label="Answer"),
    title="📘 PDF Q&A Chatbot",
    description="Ask questions about your uploaded PDF. Answers include page references."
)

# Launch the chatbot
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7757dcb3b0f6e53a24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
